### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
* If you would like to contribute to this notebook, make changes on it in useful_notebooks folder, run "Restart and Clear Output" before commit.

#### Unexpected Expansion Cases
###### Experiment set fetches unrealated labs and awards
Sometimes a fetch will get some unrelated labs and awards, this is because of the multiple awards a lab can have. This multiple awards are visited, which have users linked to them. This users also have labs, so here you go. Hopefully all are released/current already.

###### Experiment set fetches unrealted biosmaple/experiment/set
This was so far because of the experiment and biosample relation field, or the references field that links to a publication. If you ignore these fields, if should be fine.
`ignore_field = ['experiment_relation', 'biosample_relation', 'references']`

In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('koray_data')

# Please Modify the following accordingly 
# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = False

# Which status to change
change_status = 'released'

sets_in_scope = [] # ['4DNACCCC', '4DNACCCCC']
search_url = "/search/?award.project=4DN&experiments_in_set.experiment_type=TSA-seq&experimentset_type=replicate&type=ExperimentSetReplicate"  # "/search/type=AAAAAAAA"


if sets_in_scope:
    set_to_release = [ff_utils.get_metadata(i, my_auth)['uuid'] for i in sets_in_scope]
elif search_url:
    set_to_release = [i['uuid'] for i in ff_utils.search_metadata(search_url, my_auth)]

store={}
item_uuids=[]
store, uuids = ff_utils.expand_es_metadata(set_to_release, my_auth, store_frame='object',add_pc_wfr=True, ignore_field = ['experiment_relation', 'biosample_relation', 'references'])

print(len(store['experiment_set_replicate']), 'exp sets for status change')
print(len(uuids), 'items collected')
time2 = time.time()
print(round((time2-time1), 1), 'sec for collection')

28 exp sets for status change
663 items collected
5.9 sec for collection


In [13]:
for wf in set([i['display_title'].split(' run')[0] for i in store.get('workflow_run_awsem')]):
           print('    ' + wf)
        
rep_runs = [i for i in store['workflow_run_awsem'] if i['display_title'].startswith('repliseq')]
print(len(rep_runs))

    File Provenance Tracking Workflow
    repliseq-parta v16
    md5 0.2.6
    fastqc-0-11-4-1 0.2.0
44


In [14]:
def fetch_wfr_associated(wfr_uuid, my_key):
    """Given wfr_uuid, find associated output files and qcs"""
    wfr_as_list = []
    wfr_info = ff_utils.get_metadata(wfr_uuid, my_key)
    wfr_as_list.append(wfr_info['uuid'])
    if wfr_info.get('output_files'):
        for o in wfr_info['output_files']:
                if o.get('value'):
                    wfr_as_list.append(o['value']['uuid'])
                if o.get('value_qc'):
                    wfr_as_list.append(o['value_qc']['uuid']) # this is a @id
    if wfr_info.get('output_quality_metrics'):
        for qc in wfr_info['output_quality_metrics']:
            if qc.get('value'):
                wfr_as_list.append(qc['value']['uuid'])
    return wfr_as_list

ass = []
for i in rep_runs:
    result = []
    result = fetch_wfr_associated(i['uuid'], my_auth)
    ass.extend(result)
print(len(ass))

176


In [16]:
print(ass)

['ec2e8dad-6901-41a3-838b-f15187d8f853', '3fe1e603-4c38-47bb-8544-149f702f1ac9', '37fa807b-559a-4716-825d-1bbbc2e7dc9b', '00c9b144-9cfa-4b9b-918d-ed8e320b50c7', 'e10765e2-ebaf-40c8-ac7d-a9ce006fff0b', '31a25232-884f-48fc-979b-463eb6e8a2dd', 'd2745e28-8536-450b-9fbb-e9d0ff92fbed', '47e18ae7-a426-4f00-9e6d-b69234850aa4', 'd8a81e9e-ad34-4027-a443-e05fc77a2023', '259d63ae-96ef-4f40-81e9-bd794c4332da', '3492cc42-7559-4ebe-8fdf-cd01f76ee3d6', 'cc356cf3-cee9-4aad-9334-7caa4f2d06f5', 'c5204a0c-6441-4608-8b2c-4a4140ad1f52', 'f4526fe0-f0e6-41c5-81ba-e4a7d221b0ef', 'a5e3150a-a8fc-488c-af66-e7235a38e23a', 'b23efeb7-a1f9-4029-81f4-13212265ad94', '8e77bed2-bc74-4de4-89ec-52e50d35e986', '6dbd2a21-54c2-4135-b32a-c8a2794b2eee', 'ae20afa3-3f66-4794-a415-4451e05302f7', 'fd9a037b-ddc9-4086-8339-10163fb6121d', '7df56644-0b22-4fee-8375-c75a63bca7f5', 'c774e6d0-d972-4c94-b425-d90dd6b12dfc', '9a25c70b-b34d-4b62-8bf3-b01b2509ea22', 'cbbed9f2-cdb0-485f-9229-ea26bb4fb33d', '7add6514-7b36-47cd-90d2-02d04e8ed3bf',

In [19]:
resp =  ff_utils.get_es_metadata(ass, chunk_size=1000, key=my_auth)
for i in resp:
    p = i['object']
    print(p['@id'], p['status'])
    ff_utils.patch_metadata({'status': 'revoked'}, p['uuid'], my_auth)

/workflow-runs-awsem/f2c046d4-c492-4840-8c56-5f9a0afba325/ obsolete
/workflow-runs-awsem/ef4ae35f-9324-4620-9b59-76d15cf450dd/ obsolete
/workflow-runs-awsem/ec2e8dad-6901-41a3-838b-f15187d8f853/ obsolete
/workflow-runs-awsem/e3a1b803-ce07-4bff-9a78-6e29c1fabbc5/ obsolete
/workflow-runs-awsem/e10765e2-ebaf-40c8-ac7d-a9ce006fff0b/ obsolete
/workflow-runs-awsem/d8a81e9e-ad34-4027-a443-e05fc77a2023/ obsolete
/workflow-runs-awsem/d259651d-9d2d-492f-af7d-b1ca45b76ef9/ obsolete
/workflow-runs-awsem/cbddfde6-97ee-4649-b774-0d1cfc93a6bd/ obsolete
/workflow-runs-awsem/ca35f4e0-7809-4567-9b98-64767f3e5362/ obsolete
/workflow-runs-awsem/c5204a0c-6441-4608-8b2c-4a4140ad1f52/ obsolete
/workflow-runs-awsem/b99b7355-58ab-4c3f-8f99-e4058acee00e/ obsolete
/workflow-runs-awsem/b4f054df-7da1-4dbd-9136-dd2ff7fa8c0a/ obsolete
/workflow-runs-awsem/b2be5f49-ccdb-4bfa-83cc-233286469f29/ obsolete
/workflow-runs-awsem/ae7192fe-ba76-4bb4-9d96-8f5c4ad587e4/ obsolete
/workflow-runs-awsem/ac634f4a-24b0-44a0-a3cd-774

/files-processed/4DNFIWBYPDFA/ uploaded
/files-processed/4DNFISU4LHRC/ uploaded
/files-processed/4DNFI9MXQ1D2/ uploaded
/files-processed/4DNFIXA5SPUF/ uploaded
/files-processed/4DNFI4NCIO78/ uploaded
/files-processed/4DNFIBKQM2HS/ uploaded
/files-processed/4DNFI9UFPMRW/ uploaded
/files-processed/4DNFISVDPKK4/ uploaded
/files-processed/4DNFI7MVLEKI/ uploaded
/files-processed/4DNFIADYR67E/ uploaded
/files-processed/4DNFIMJWHO2Q/ uploaded
/files-processed/4DNFIDXDKGWI/ uploaded
/files-processed/4DNFI9DCUW4W/ uploaded
/files-processed/4DNFIIQILCJK/ uploaded
/files-processed/4DNFI2NWMFAN/ uploaded
/files-processed/4DNFI9N8MOE4/ uploaded
/files-processed/4DNFITV9Z1G1/ uploaded
/files-processed/4DNFIFGIQ7DH/ uploaded
/files-processed/4DNFIL9RZAAU/ uploaded
/files-processed/4DNFIJDPCDA6/ uploaded
/files-processed/4DNFI6P1W8MR/ uploaded
/files-processed/4DNFIXL8CZAN/ uploaded
/files-processed/4DNFIYJ6SENN/ uploaded
/files-processed/4DNFITIXI2U9/ uploaded
/files-processed/4DNFI4K2BZ9F/ uploaded


In [9]:
# TODO
# Check audits

# to decide which items to ignore as they have a 'higher' status
STATUS_LEVEL = {
    # standard_status
    "released": 10, "current": 10,
    "released to project": 9,
    "pre-release": 8,
    "planned": 6, "submission in progress": 6,
    "in review by lab": 4,
    "revoked": 0, "archived": 0,"deleted": 0, "obsolete": 0, "replaced": 0, "archived to project": 0,
    # additional file statuses
    'to be uploaded by workflow': 4, 'uploading': 4, 'uploaded': 4, 'upload failed': 4, 'draft': 4, 'released to lab': 4}

change_level = STATUS_LEVEL.get(change_status, 1)

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = [i for i in store if i.startswith('experiment') and not i.startswith('experiment_set')]
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(all_exps), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum'):
        print(a_file['accession'], 'missing content md5 sum')
    if not a_file.get('md5sum'):
        print(a_file['accession'], 'md5 was not calculated during upload, missing md5sum')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

# check processed files
print('\nFILE PROCESSED CHECK')
if store.get('file_processed'):
    for a_file in store['file_processed']:
        if a_file['file_format'] == '/file-formats/pairs/':
            if not a_file.get('quality_metric'):
                print(a_file['accession'], 'missing Pairsqc')
        if not a_file.get('source_experiments'):
            print(a_file['accession'], 'user submitted or produced by sbg runs')
        if check_wfrs:
            dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
# list all wf types found
print('  Following run types are found:')
for wf in set([i['display_title'].split(' run')[0] for i in store.get('workflow_run_awsem')]):
           print('    ' + wf)
if store.get('workflow_run_awsem'):
    for wfr in store['workflow_run_awsem']:
        if wfr['run_status'] != 'complete':
            print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i, x['uuid'], x['status']] for x in store[i] if STATUS_LEVEL.get(x['status']) == 0]
    if weird:
        for case in weird:
            print(case)
        print()

EXPSET CHECK
4DNESD1KO9LC missing processing tag Set of Input for No1 control for LaminA/C TSA-seq 
4DNESURW4RWH missing processing tag Replicates of RNA Pol II TSA-seq version 1 Reactio
4DNES19URNAH missing processing tag Replicates of LaminA/C TSA-seq version 1 Reaction 
4DNES27KAIML missing processing tag Set of Input for SON Ab2 TSA-seq version 1  Reacti
4DNESVTOWO4Z missing processing tag Set of Input for No1 control for RNA Pol II TSA-se
4DNESWB19384 missing processing tag Replicates of No1 control for LaminB TSA-seq versi
4DNESNZGPYE6 missing processing tag Replicates of No1 Control for pSC35 TSA-seq versio
4DNESNER8V11 missing processing tag Replicates of pSC35 TSA-seq version 1 Reaction Con
4DNESVZYLFAN missing processing tag Replicates of No1 control for LaminA/C TSA-seq ver
4DNESOSG1ECS missing processing tag Set of Input for No1 Control for SON TSA-seq versi
4DNESQBBEUUS missing processing tag Replicates of No1 control for LaminA/C TSA-seq ver
4DNES3IB82SC missing processin

In [5]:
# Check status
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} needs change'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))
    
       

biosource                 Out of 1, 0 skipped, 1 matching, 0 unusual, and 0 needs change
protocol                  Out of 26, 0 skipped, 1 matching, 0 unusual, and 25 needs change
award                     Out of 4, 0 skipped, 4 matching, 0 unusual, and 0 needs change
organism                  Out of 1, 0 skipped, 1 matching, 0 unusual, and 0 needs change
ontology_term             Out of 15, 0 skipped, 15 matching, 0 unusual, and 0 needs change
user                      Out of 13, 0 skipped, 13 matching, 0 unusual, and 0 needs change
antibody                  Out of 8, 0 skipped, 2 matching, 0 unusual, and 6 needs change
target                    Out of 5, 0 skipped, 5 matching, 0 unusual, and 0 needs change
lab                       Out of 5, 0 skipped, 5 matching, 0 unusual, and 0 needs change
software                  Out of 11, 0 skipped, 11 matching, 0 unusual, and 0 needs change
file_processed            Out of 217, 0 skipped, 0 matching, 0 unusual, and 217 needs change
file_refe

In [ ]:
# If you want to patch the status, change action to True
action = False

reviewed = ""
reviewed = input('Did another wrangler review this release? (y/n):')
if reviewed != 'y':
    raise KeyError('A key step is missing!')
    
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
            if action:
                patch_data = {'status': change_status}
                if change_status == 'released' and a_type in ['publication']:
                    patch_data = {'status': 'current'}
                   
                ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth) 

        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} UPDATED with status'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))